# Project 2 Regression Challenge
## Daniel Rossetti

# 1 - Import Libraries and Read Data

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [3]:
# Read Data
train = pd.read_csv('../datasets/train.csv')
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [26]:
print(train.shape)
train_nulls = pd.DataFrame(train.isnull().sum())
train_nulls[0]
train_nulls[train_nulls[0] != 0]

(2051, 81)


,0
Lot Frontage,330
Alley,1911
Mas Vnr Type,22
Mas Vnr Area,22
Bsmt Qual,55
Bsmt Cond,55
Bsmt Exposure,58
BsmtFin Type 1,55
BsmtFin SF 1,1
BsmtFin Type 2,56


In [28]:
test = pd.read_csv('../datasets/test.csv')
test_nulls = pd.DataFrame(test.isnull().sum())
test_nulls[test_nulls[0] != 0]

,0
Lot Frontage,160
Alley,820
Mas Vnr Type,1
Mas Vnr Area,1
Bsmt Qual,25
Bsmt Cond,25
Bsmt Exposure,25
BsmtFin Type 1,25
BsmtFin Type 2,25
Electrical,1


In [36]:
nulls = pd.DataFrame()
nulls['train'], nulls['test'] = train_nulls, test_nulls
nulls[(nulls['train'] != 0) | (nulls['test'] != 0)]

,train,test
Lot Frontage,330,160.0
Alley,1911,820.0
Mas Vnr Type,22,1.0
Mas Vnr Area,22,1.0
Bsmt Qual,55,25.0
Bsmt Cond,55,25.0
Bsmt Exposure,58,25.0
BsmtFin Type 1,55,25.0
BsmtFin SF 1,1,0.0
BsmtFin Type 2,56,25.0
